In [1]:
import pandas as pd
import numpy as np
import feather
import pymc3 as pm

In [2]:
texts_data = pd.read_csv("/Users/Hun/hse_thesis/region_topics.csv")

In [ ]:
texts_data.to_excel("/Users/Hun/hse_thesis/region_topics_pd.xlsx")

In [4]:
min(texts_data["topic_1"])

8.2406262875999997e-06

In [7]:
texts_data.round(8).to_csv("/Users/Hun/hse_thesis/region_topics_pd.csv")

In [3]:
texts_data.head()

,author_id,region,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_140,topic_141,topic_142,topic_143,topic_144,topic_145,topic_146,topic_147,topic_148,topic_149
0,246087682,Северная Осетия - Алания,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,...,0.003536,0.000022,0.000022,0.000022,0.004591,0.000022,0.000022,0.000022,0.010027,0.000022
1,246087682,Северная Осетия - Алания,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,...,0.003074,0.004040,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018
2,147429131,Волгоградская область,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,...,0.000022,0.007591,0.006936,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.006940
3,147429131,Волгоградская область,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,...,0.000022,0.034494,0.000022,0.000022,0.000022,0.000022,0.000022,0.000022,0.009842,0.006776
4,147429131,Волгоградская область,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,...,0.000021,0.012678,0.008308,0.119572,0.000021,0.000021,0.000021,0.000021,0.017864,0.000021


In [15]:
region_names = texts_data.region.unique()
n_regions = len(texts_data.region.unique())
region_idx = texts_data['region'].values

In [14]:
texts_data['region'].values

array(['Северная Осетия - Алания', 'Северная Осетия - Алания',
       'Волгоградская область', ..., 'Москва', 'Хабаровский край',
       'Хабаровский край'], dtype=object)

In [17]:
with pm.Model() as hierarchical_model:
    # Hyperpriors for group nodes
    mu_a = pm.Normal('mu_alpha', mu=0., sd=100**2)
    sigma_a = pm.Uniform('sigma_alpha', lower=0, upper=100)
    mu_b = pm.Normal('mu_beta', mu=0., sd=100**2)
    sigma_b = pm.Uniform('sigma_beta', lower=0, upper=100)
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    a = pm.Normal('alpha', mu=mu_a, sd=sigma_a, shape=n_regions)
    # Intercept for each county, distributed around group mean mu_a
    b = pm.Normal('beta', mu=mu_b, sd=sigma_b, shape=n_regions)
    
    # Model error
    eps = pm.Uniform('eps', lower=0, upper=100)
    
    # Model prediction of radon level
    # a[county_idx] translates to a[0, 0, 0, 1, 1, ...],
    # we thus link multiple household measures of a county
    # to its coefficients.
    a[region_idx]
    radon_est = a[region_idx] + b[region_idx] * data.region.values
    
    # Data likelihood
    radon_like = pm.Normal('radon_like', mu=radon_est, sd=eps, observed=data.topic_130)

Applied interval-transform to sigma_alpha and added transformed sigma_alpha_interval to model.
Applied interval-transform to sigma_beta and added transformed sigma_beta_interval to model.
Applied interval-transform to eps and added transformed eps_interval to model.


AsTensorError: ("Cannot convert ['Северная Осетия - Алания' 'Северная Осетия - Алания'\n 'Волгоградская область' ..., 'Москва' 'Хабаровский край'\n 'Хабаровский край'] to TensorType", <class 'numpy.ndarray'>)

In [ ]:
with pm.Model() as model:
    # specify glm and pass in data. The resulting linear model, its likelihood and 
    # and all its parameters are automatically added to our model.
    pm.glm.glm('topic_130 ~ region', texts_data, family=pm.glm.families.Normal())
    start = pm.find_MAP() # Find starting value by optimization
    step = pm.NUTS(scaling=start) # Instantiate MCMC sampling algorithm
    trace = pm.sample(10, step, start=start, progressbar=True) # draw 2000 posterior samples using NUTS sampling

Applied log-transform to sd and added transformed sd_log to model.
